In [ ]:
import numpy as np
import pandas as pd

# Matplotlib forms basis for visualization in Python
import matplotlib.pyplot as plt
from scipy import stats

# We will use the Seaborn library
import seaborn as sns
sns.set()

#precisao de casas decimais
pd.set_option("display.precision", 3)
#maximo de colunas
pd.set_option("display.max_columns", 200)
#maximo de linhas
pd.set_option("display.max_rows", 2000)

In [ ]:
df = pd.read_csv('./incident_event_log.csv', low_memory=False)

In [ ]:
#ajustando notify
df['notify'] = df['notify'].map({'Do Not Notify': False, 'Send Email': True})

In [ ]:
#ajustando datas
df['opened_at'] = pd.to_datetime(df['opened_at'])
df['sys_created_at'] = pd.to_datetime(df['sys_created_at'])
df['sys_updated_at'] = pd.to_datetime(df['sys_updated_at'])
df['resolved_at'] = pd.to_datetime(df['resolved_at'])
df['closed_at'] = pd.to_datetime(df['closed_at'])

In [ ]:
#ajustando categorias
df['location'] = df['location'].str.lstrip('Location ').astype('float')
df['category'] = df['category'].str.lstrip('Category ').astype('float')
df['subcategory'] = df['subcategory'].str.lstrip('Subcategory ').astype('float')
df['u_symptom'] = df['u_symptom'].str.lstrip('Symptom ').astype('float')
df['closed_code'] = df['closed_code'].str.lstrip('code ').astype('float')
df['problem_id'] = df['problem_id'].str.lstrip('Problem ID ').astype('float')

In [ ]:
#ajustando impacto
df['impact'] = df['impact'].str.partition('-')[0].astype('float')
df['urgency'] = df['urgency'].str.partition('-')[0].astype('float')
df['priority'] = df['priority'].str.partition('-')[0].astype('float')

In [ ]:
#ajustando id pessoas
df['resolved_by'] = df['resolved_by'].str.lstrip('Resolved by ').astype('float')
df['assignment_group'] = df['assignment_group'].str.lstrip('Group ').astype('float')
df['assigned_to'] = df['assigned_to'].str.lstrip('Resolver ').astype('float')
df['sys_updated_by'] = df['sys_updated_by'].str.lstrip('Updated by ').astype('float')
df['sys_created_by'] = df['sys_created_by'].str.lstrip('Created by ').astype('float')
df['opened_by'] = df['opened_by'].str.lstrip('Opened by ').astype('float')
df['caller_id'] = df['caller_id'].str.lstrip('Caller ').astype('float')

In [ ]:
#diferença entre abertura e fechamento 
df['days_to_close'] = df['closed_at'] - df['opened_at']
df['days_to_resolve'] = df['resolved_at'] - df['opened_at']
df['num_days_to_close'] = df['days_to_close'] / pd.Timedelta(1, 'D')
df['num_days_to_resolve'] = df['days_to_resolve'] / pd.Timedelta(1, 'D')

In [ ]:
df.head(200)

In [ ]:
#tipos de abertura
df[df['sys_mod_count'] == 0].groupby(by='incident_state').size()

In [ ]:
#verificamos que possuem mais chamados fechados que chamados abertos
#O que pode indicar que alguns chamados foram registrados apenas para histórico, porque ja iniciaram com status CLOSED.
#len(pd.unique(df[df['sys_mod_count'] == 0]['number']))

df[df['sys_mod_count'] == 0].groupby(by='incident_state').size().sum() - len(df[df['incident_state'] == 'Closed'])

In [ ]:
#quantia de chamados com datas inválidas
#O resultado aponta que pode haver dados corrompidos/inconsistentes, pois as datas estão negativas ou o chamado foi registrado em um período após dua resolução. 
print (len(df[df['num_days_to_close'] < 0]))
print (len(df[df['num_days_to_resolve'] < 0]))

In [ ]:
#quantia de chamados que iniciam com alterações
df[((df['sys_mod_count'] != 0) & (df['sys_created_at'] == df['sys_updated_at']))].groupby(by='number').size()

In [ ]:
#remover qualquer chamado com periodo para fechamento negativo
#limpando o dataset para obter informações mais concisas.
df = df[((df['num_days_to_close'] > 0) & (df['num_days_to_resolve'] > 0))]

In [ ]:
#chamados fechados - mostrando que o ciclo de abertura e fechamento está incompleto ou seja não são todos os chamados que conseguem ser fechados.
closed_df = df[df['incident_state'] == 'Closed']

In [ ]:
#avaliação do período que levou para fechar os tickets.
closed_df['days_to_close'].describe()

In [ ]:
#avaliação do período que levou para resolver os tickets.
closed_df['days_to_resolve'].describe()

In [ ]:
category_df = closed_df.groupby(by=['category', 'subcategory'])
category_df.size()

In [ ]:
#chamados mais recorrentes
#Analise feita a fim de concluir se a complexidade dos chamados estão relacionados a area que foram abertos.
category_df.size().sort_values(ascending=False).head(10)

In [ ]:
category_df['days_to_close'].describe()

In [ ]:
#selecionando colunas que se repetem nas linhas referentes ao mesmo incidente
common_columns = np.array(['number', 'caller_id', 'opened_by', 'opened_at', 'sys_created_by', 'sys_created_at', 'contact_type', 'notify', 'closed_code', 'resolved_at', 'closed_at'])
#agrupando em um novo df
grouped_df = df.groupby(by=list(common_columns))

In [ ]:
plt.figure(figsize=(15,6))
_ = sns.distplot(closed_df['num_days_to_close'], fit=stats.norm)

In [ ]:
plt.figure(figsize=(15,6))
_ = sns.distplot(closed_df['num_days_to_resolve'], fit=stats.norm)

In [ ]:
#com o gráfico a seguir, conseguimos perceber que o impacto afeta o tempo para o chamado ser concluido
sns.pairplot(closed_df,
             y_vars=['num_days_to_close', 'num_days_to_resolve'],
             x_vars=['subcategory', 'location', 'resolved_by'],
             dropna=True,
             hue='impact');

In [ ]:
#com o gráfico a seguir, conseguimos perceber que a urgência afeta o tempo para o chamado ser concluido
sns.pairplot(closed_df,
             y_vars=['num_days_to_close', 'num_days_to_resolve'],
             x_vars=['subcategory', 'location', 'resolved_by'],
             dropna=True,
             hue='urgency');

In [ ]:
#o tipo de resolução se relaciona com o tempo de conclusão
sns.jointplot(x='num_days_to_resolve',
              y='closed_code', 
              data=closed_df);

In [ ]:
sns.jointplot(x='subcategory',
              y='closed_code', 
              data=closed_df);

In [ ]:
plt.figure(figsize=(15,6))
fig = sns.countplot(x='closed_code', hue='priority', data=closed_df);
for item in fig.get_xticklabels():
    item.set_rotation(90)
_ = plt.legend(loc='upper right')

In [ ]:
#Porcentagem de chamados que foram rejeitados, erc mto pequeno n vai impactar na media de dias para finalizar um chamado 
closed_df[closed_df['reopen_count'] > 0].groupby('number').size().sum() / closed_df.groupby('number').size().sum()

In [ ]:
# Adicionando um campo para verificar quantos incidentes estão abertos no dia de abertura de um incidente
closed_gr_df = df[df['incident_state'] == 'Closed'].copy()
closed_gr_df['opened_at'] = pd.to_datetime(closed_gr_df['opened_at'] ).dt.date
closed_gr_df['resolved_at'] = pd.to_datetime(closed_gr_df['resolved_at'] ).dt.date
closed_gr_df['closed_at'] = pd.to_datetime(closed_gr_df['closed_at'] ).dt.date
open_incidents = []
for row in closed_gr_df.iterrows():
    total = len(closed_gr_df[(closed_gr_df['opened_at'] <= row[1]['opened_at']) & (closed_gr_df['closed_at'] >= row[1]['opened_at'])])
    open_incidents.append(total)
closed_gr_df['open_incidents'] = open_incidents 

In [ ]:
#Nesse gráfico podemos ver a relação entre a quantidade de dias que um incidente demora para ser fechado pelo número 
# de incidentes ativos no dia de abertura do mesmo... Conseguimos notar que o numero de incidentes abertos não interfere
# na quantidade de dias para fechar um incidente
temp = closed_gr_df.sort_values(by=['opened_at']).groupby(by=['opened_at', 'priority']).mean()
ax_qtd = temp['num_days_to_close'].plot(figsize=(20,10), kind='bar', stacked=True).set_ylabel('num_days_to_close')
temp['open_incidents'].plot(secondary_y=True).set_ylabel('open_incidents')

fig = ax_qtd.get_figure()
ax_qtd = fig.get_axes()
ax_qtd[1].set_ylim(0,6000)

plt.gca().set_xticklabels('')
plt.show()